In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import env
import numpy as np
import acquire
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import prepare


In [5]:
def prep_zillow(df):
    # drop redundant id code columns
    id_cols = [col for col in df.columns if 'typeid' in col or col in ['id', 'parcelid']]
    df = df.drop(columns=id_cols)
    # filter for single family properties
    df = df[df.propertylandusedesc == 'Single Family Residential']
    # drop specified columns
    cols_to_drop = ['calculatedbathnbr',
                    'finishedfloor1squarefeet',
                    'finishedsquarefeet12', 
                    'regionidcity',
                    'landtaxvaluedollarcnt',
                    'taxamount',
                    'rawcensustractandblock',
                    'roomcnt',
                    'regionidcounty']
    df = df.drop(columns=cols_to_drop)
    # fill null values with 0 in specified columns
    cols_to_fill_zero = ['fireplacecnt',
                         'garagecarcnt',
                         'garagetotalsqft',
                         'hashottuborspa',
                         'poolcnt',
                         'threequarterbathnbr',
                         'taxdelinquencyflag']
    for col in cols_to_fill_zero:
        df[col] = np.where(df[col].isna(), 0, df[col]) 
    # drop columns with more than 5% null values
    for col in df.columns:
        if df[col].isnull().mean() > .05:
            df = df.drop(columns=col)
    # drop rows that remain with null values
    df = df.dropna()   
    # changing numeric codes to strings
    df['fips'] = df.fips.apply(lambda fips: '0' + str(int(fips)))
    df['regionidzip'] = df.regionidzip.apply(lambda x: str(int(x)))
    df['censustractandblock'] = df.censustractandblock.apply(lambda x: str(int(x)))
    # change the 'Y' in taxdelinquencyflag to 1
    df['taxdelinquencyflag'] = np.where(df.taxdelinquencyflag == 'Y', 1, df.taxdelinquencyflag)
    # change boolean column to int
    df['hashottuborspa'] = df.hashottuborspa.apply(lambda x: str(int(x)))
    # changing year from float to int
    df['yearbuilt'] = df.yearbuilt.apply(lambda x: int(x))
    df['assessmentyear'] = df.yearbuilt.apply(lambda x: int(x))
    # adding a feature: age 
    df['age'] = 2017 - df.yearbuilt
    # add a feature: has_garage
    df['bool_has_garage'] = np.where(df.garagecarcnt > 0, 1, 0)
    # add a feature: has_pool
    df['bool_has_pool'] = np.where(df.poolcnt > 0, 1, 0)
    # add a feature: has_fireplace
    df['bool_has_fireplace'] = np.where(df.fireplacecnt > 0, 1, 0)
    # adding prefix to boolean columns
    df = df.rename(columns={'hashottuborspa': 'bool_hashottuborspa'})
    df = df.rename(columns={'taxdelinquencyflag': 'bool_taxdelinquencyflag'})
    # rename sqft column
    df = df.rename(columns={'calculatedfinishedsquarefeet': 'sqft'})

    return df

In [6]:
df = acquire.zillow_data()

Reading from local CSV...


In [7]:
df = prep_zillow(df)

In [8]:
def train_validate_test_split(df, test_size=.2, validate_size=.3, random_state=42):
    '''
    This function takes in a dataframe, then splits that dataframe into three separate samples
    called train, test, and validate, for use in machine learning modeling.

    Three dataframes are returned in the following order: train, test, validate. 
    
    The function also prints the size of each sample.
    '''
    # split the dataframe into train and test
    train, test = train_test_split(df, test_size=.2, random_state=42)
    # further split the train dataframe into train and validate
    train, validate = train_test_split(train, test_size=.3, random_state=42)
    # print the sample size of each resulting dataframe
    print(f'train\t n = {train.shape[0]}')
    print(f'test\t n = {test.shape[0]}')
    print(f'validate n = {validate.shape[0]}')

    return train, validate, test

In [9]:
train, validate, test = train_validate_test_split(df)

train	 n = 29001
test	 n = 10358
validate n = 12429


In [10]:
target = 'logerror'

In [11]:
def remove_outliers(train, validate, test, k, col_list):
    ''' 
    This function takes in a dataset split into three sample dataframes: train, validate and test.
    It calculates an outlier range based on a given value for k, using the interquartile range 
    from the train sample. It then applies that outlier range to each of the three samples, removing
    outliers from a given list of feature columns. The train, validate, and test dataframes 
    are returned, in that order. 
    '''
    # Create a column that will label our rows as containing an outlier value or not
    train['outlier'] = False
    validate['outlier'] = False
    test['outlier'] = False
    for col in col_list:

        q1, q3 = train[col].quantile([.25, .75])  # get quartiles
        
        iqr = q3 - q1   # calculate interquartile range
        
        upper_bound = q3 + k * iqr   # get upper bound
        lower_bound = q1 - k * iqr   # get lower bound

        # update the outlier label any time that the value is outside of boundaries
        train['outlier'] = np.where(((train[col] < lower_bound) | (train[col] > upper_bound)) & (train.outlier == False), True, train.outlier)
        validate['outlier'] = np.where(((validate[col] < lower_bound) | (validate[col] > upper_bound)) & (validate.outlier == False), True, validate.outlier)
        test['outlier'] = np.where(((test[col] < lower_bound) | (test[col] > upper_bound)) & (test.outlier == False), True, test.outlier)

    # remove observations with the outlier label in each of the three samples
    train = train[train.outlier == False]
    train = train.drop(columns=['outlier'])

    validate = validate[validate.outlier == False]
    validate = validate.drop(columns=['outlier'])

    test = test[test.outlier == False]
    test = test.drop(columns=['outlier'])

    # print the remaining 
    print(f'train\t n = {train.shape[0]}')
    print(f'test\t n = {test.shape[0]}')
    print(f'validate n = {validate.shape[0]}')

    return train, validate, test

In [12]:
outlier_columns = [col for col in df.columns if ((df[col].dtype != 'object') & (col not in [target, 'latitude', 'longitude']))]
train, validate, test = remove_outliers(train, validate, test, 3, outlier_columns)

train	 n = 16969
test	 n = 6144
validate n = 7247


**Note** I think this removes entirely too much data (train from n=29001 to n=16969). We should probably find a more nuanced way to handle outliers (perhaps leave them in for some columns). But for now, we will drop them all and move on.

In [13]:
def scale_zillow(train, validate, test, target, scaler_type=MinMaxScaler()):
    '''
    This takes in the train, validate, and test dataframes, as well as the target label. 

    It then fits a scaler object to the train sample based on the given sample_type, applies that
    scaler to the train, validate, and test samples, and appends the new scaled data to the 
    dataframes as additional columns with the prefix 'scaled_'. 

    train, validate, and test dataframes are returned, in that order. 
    '''
    # identify quantitative features to scale
    quant_features = [col for col in train.columns if (train[col].dtype != 'object') 
                                                    & (col != target) 
                                                    & ('bool_' not in col)]
    # establish empty dataframes for storing scaled dataset
    train_scaled = pd.DataFrame(index=train.index)
    validate_scaled = pd.DataFrame(index=validate.index)
    test_scaled = pd.DataFrame(index=test.index)
    # screate and fit the scaler
    scaler = scaler_type.fit(train[quant_features])
    # adding scaled features to scaled dataframes
    train_scaled[quant_features] = scaler.transform(train[quant_features])
    validate_scaled[quant_features] = scaler.transform(validate[quant_features])
    test_scaled[quant_features] = scaler.transform(test[quant_features])
    # add 'scaled' prefix to columns
    for feature in quant_features:
        train_scaled = train_scaled.rename(columns={feature: f'scaled_{feature}'})
        validate_scaled = validate_scaled.rename(columns={feature: f'scaled_{feature}'})
        test_scaled = test_scaled.rename(columns={feature: f'scaled_{feature}'})
    # concat scaled feature columns to original train, validate, test df's
    train = pd.concat([train, train_scaled], axis=1)
    validate = pd.concat([validate, validate_scaled], axis=1)
    test = pd.concat([test, test_scaled], axis=1)

    return train, validate, test

In [14]:
train, validate, test = scale_zillow(train, validate, test, target)

In [15]:
df[[col for col in df.columns if df[col].dtype == 'object']].nunique()

fips                             3
bool_hashottuborspa              2
propertycountylandusecode       22
regionidzip                    379
bool_taxdelinquencyflag          2
censustractandblock          31172
propertylandusedesc              1
dtype: int64

In [16]:
def encode_zillow(train, validate, test, target):
    '''
    This function takes in the train, validate, and test samples, as well as a label for the target variable. 

    It then encodes each of the categorical variables using one-hot encoding with dummy variables and appends 
    the new encoded variables to the original dataframes as new columns with the prefix 'enc_{variable_name}'.

    train, validate and test dataframes are returned (in that order)
    '''
    # identify the features to encode (categorical features represented by non-numeric data types)
    features_to_encode = [col for col in train.columns if (train[col].dtype == 'object') 
                                                        & ('bool_' not in col) 
                                                        & (col != target)
                                                        & (train[col].nunique() < 25)]
    #iterate through the list of features                  
    for feature in features_to_encode:
        # establish dummy variables
        dummy_df = pd.get_dummies(train[feature],
                                  prefix=f'enc_{train[feature].name}',
                                  drop_first=True)
        # add the dummies as new columns to the original dataframe
        train = pd.concat([train, dummy_df], axis=1)

    # then repeat the process for the other two samples:

    for feature in features_to_encode:
        dummy_df = pd.get_dummies(validate[feature],
                                  prefix=f'enc_{validate[feature].name}',
                                  drop_first=True)
        validate = pd.concat([validate, dummy_df], axis=1)
        
    for feature in features_to_encode:
        dummy_df = pd.get_dummies(test[feature],
                                  prefix=f'enc_{test[feature].name}',
                                  drop_first=True)
        test = pd.concat([test, dummy_df], axis=1)
    
    return train, validate, test

In [17]:
train, validate, test = encode_zillow(train, validate, test, target)

In [18]:
train.head()

,bathroomcnt,bedroomcnt,sqft,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,bool_hashottuborspa,latitude,longitude,lotsizesquarefeet,poolcnt,propertycountylandusecode,regionidzip,threequarterbathnbr,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,bool_taxdelinquencyflag,censustractandblock,logerror,propertylandusedesc,age,bool_has_garage,bool_has_pool,bool_has_fireplace,scaled_bathroomcnt,scaled_bedroomcnt,scaled_sqft,scaled_fireplacecnt,scaled_fullbathcnt,scaled_garagecarcnt,scaled_garagetotalsqft,scaled_latitude,scaled_longitude,scaled_lotsizesquarefeet,scaled_poolcnt,scaled_threequarterbathnbr,scaled_yearbuilt,scaled_structuretaxvaluedollarcnt,scaled_taxvaluedollarcnt,scaled_assessmentyear,scaled_age,enc_fips_06059,enc_fips_06111,enc_propertycountylandusecode_0101,enc_propertycountylandusecode_0102,enc_propertycountylandusecode_0103,enc_propertycountylandusecode_0104,enc_propertycountylandusecode_0108,enc_propertycountylandusecode_0109,enc_propertycountylandusecode_010G,enc_propertycountylandusecode_010M,enc_propertycountylandusecode_1,enc_propertycountylandusecode_1110,enc_propertycountylandusecode_1111,enc_propertycountylandusecode_122
10044,1.0,2.0,816.0,06037,0.0,1.0,0.0,0.0,0,33985475.0,-118249143.0,5849.0,0.0,0100,95982,0.0,1925,55113.0,163145.0,1925,0,60375328001007,-0.427364,Single Family Residential,92,0,0,0,0.0,0.285714,0.130273,0.0,0.0,0.0,0.0,0.468132,0.603087,0.293140,0.0,0.0,0.343066,0.083273,0.082979,0.343066,0.656934,0,0,0,0,0,0,0,0,0,0,0,0,0,0
23280,2.0,4.0,2422.0,06037,0.0,2.0,0.0,0.0,0,34608882.0,-118195545.0,16005.0,0.0,0100,97329,0.0,2002,273600.0,342000.0,2002,0,60379102091006,0.013555,Single Family Residential,15,0,0,0,0.2,0.571429,0.445360,0.0,0.2,0.0,0.0,0.920857,0.633751,0.865116,0.0,0.0,0.905109,0.414486,0.179412,0.905109,0.094891,0,0,0,0,0,0,0,0,0,0,0,0,0,0
55542,2.0,4.0,1404.0,06037,0.0,2.0,0.0,0.0,0,34187260.0,-118646355.0,7275.0,0.0,0100,96342,0.0,1960,208047.0,557565.0,1960,0,60371352031024,-0.033526,Single Family Residential,57,0,0,0,0.2,0.571429,0.245635,0.0,0.2,0.0,0.0,0.614671,0.375837,0.373451,0.0,0.0,0.598540,0.315112,0.295637,0.598540,0.401460,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7420,1.0,2.0,912.0,06037,0.0,1.0,0.0,0.0,0,33994538.0,-118318006.0,4920.0,0.0,0100,96024,0.0,1916,98821.0,197642.0,1916,0,60372346001001,-0.077108,Single Family Residential,101,0,0,0,0.0,0.285714,0.149107,0.0,0.0,0.0,0.0,0.474714,0.563689,0.240820,0.0,0.0,0.277372,0.149532,0.101579,0.277372,0.722628,0,0,0,0,0,0,0,0,0,0,0,0,0,0
68942,3.0,6.0,2913.0,06037,0.0,3.0,0.0,0.0,0,34319169.0,-118458195.0,16743.0,0.0,0100,96368,0.0,1950,89029.0,114271.0,1950,0,60371064061047,0.053264,Single Family Residential,67,0,0,0,0.4,0.857143,0.541691,0.0,0.4,0.0,0.0,0.710464,0.483486,0.906679,0.0,0.0,0.525547,0.134688,0.056628,0.525547,0.474453,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
def get_row_nulls(df):    
    df2 = pd.DataFrame()
    df2['n_rows_null'] = df.isnull().sum()
    df2['pct_rows_null'] = df.isnull().mean()
    df2 = df2.reset_index()
    df2 = df2.rename(columns={'index': 'feature'})
    return df2

def get_column_nulls(df):   
    df2 = pd.DataFrame(df.isnull().sum(axis=1))
    df2.columns = ['n_cols_null']
    df2['pct_cols_null'] = df.isnull().mean(axis=1)
    return df2

### Testing prepare.py

In [20]:
df = acquire.zillow_data()

Reading from local CSV...


In [21]:
df = prep_zillow(df)

In [22]:
target = 'logerror'

In [23]:
train, validate, test = prepare.train_validate_test_split(df)

train	 n = 29001
test	 n = 10358
validate n = 12429


In [24]:
outlier_columns = [col for col in df.columns if ((df[col].dtype != 'object') & (col not in [target, 'latitude', 'longitude']))]
train, validate, test = prepare.remove_outliers(train, validate, test, 3, outlier_columns)

train	 n = 16969
test	 n = 6144
validate n = 7247


In [25]:
train, validate, test = prepare.scale_zillow(train, validate, test, target)

In [26]:
scaled_cols = [col for col in train.columns if 'scaled_' in col]
train[scaled_cols].head()

,scaled_bathroomcnt,scaled_bedroomcnt,scaled_sqft,scaled_fireplacecnt,scaled_fullbathcnt,scaled_garagecarcnt,scaled_garagetotalsqft,scaled_latitude,scaled_longitude,scaled_lotsizesquarefeet,scaled_poolcnt,scaled_threequarterbathnbr,scaled_yearbuilt,scaled_structuretaxvaluedollarcnt,scaled_taxvaluedollarcnt,scaled_assessmentyear,scaled_age
10044,0.0,0.285714,0.130273,0.0,0.0,0.0,0.0,0.468132,0.603087,0.293140,0.0,0.0,0.343066,0.083273,0.082979,0.343066,0.656934
23280,0.2,0.571429,0.445360,0.0,0.2,0.0,0.0,0.920857,0.633751,0.865116,0.0,0.0,0.905109,0.414486,0.179412,0.905109,0.094891
55542,0.2,0.571429,0.245635,0.0,0.2,0.0,0.0,0.614671,0.375837,0.373451,0.0,0.0,0.598540,0.315112,0.295637,0.598540,0.401460
7420,0.0,0.285714,0.149107,0.0,0.0,0.0,0.0,0.474714,0.563689,0.240820,0.0,0.0,0.277372,0.149532,0.101579,0.277372,0.722628
68942,0.4,0.857143,0.541691,0.0,0.4,0.0,0.0,0.710464,0.483486,0.906679,0.0,0.0,0.525547,0.134688,0.056628,0.525547,0.474453


In [27]:
train, validate, test = prepare.encode_zillow(train, validate, test, target)

In [28]:
encoded_cols = [col for col in train.columns if 'enc_' in col]
train[encoded_cols].head()

,enc_fips_06059,enc_fips_06111,enc_propertycountylandusecode_0101,enc_propertycountylandusecode_0102,enc_propertycountylandusecode_0103,enc_propertycountylandusecode_0104,enc_propertycountylandusecode_0108,enc_propertycountylandusecode_0109,enc_propertycountylandusecode_010G,enc_propertycountylandusecode_010M,enc_propertycountylandusecode_1,enc_propertycountylandusecode_1110,enc_propertycountylandusecode_1111,enc_propertycountylandusecode_122
10044,0,0,0,0,0,0,0,0,0,0,0,0,0,0
23280,0,0,0,0,0,0,0,0,0,0,0,0,0,0
55542,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7420,0,0,0,0,0,0,0,0,0,0,0,0,0,0
68942,0,0,0,0,0,0,0,0,0,0,0,0,0,0
